In [1]:
#importing the libraries

import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
import pandas as pd
import re
import numpy as np
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import math
from sklearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# pull and clean data

In [2]:
questions = pd.read_json("200k_questions.json")

In [3]:
questions

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999


In [4]:
# clean columns
questions.columns = questions.columns.str.strip()

In [5]:
# Check and verify no missing/imcomplete data
questions[(questions.isna().any(axis=1)) & (questions['round'] != 'Final Jeopardy!') & (questions['round'] != 'Tiebreaker')]

,category,air_date,question,value,answer,round,show_number


In [6]:
questions.nunique()

category        27995
air_date         3640
question       216132
value             149
answer          88269
round               4
show_number      3640
dtype: int64

In [7]:
# Clean up Columns
questions['air_date'] = pd.to_datetime(questions['air_date'])

questions['value'] = questions['value'].str.replace("$","")
questions['value'] = questions['value'].str.replace(",","")
questions['value'] = questions['value'].fillna('0')
questions['value'] = questions['value'].astype(int)

In [8]:
questions = questions[questions['question'].str.contains('www')==False].reset_index(drop=True)
questions = questions[questions['category'].str.contains('missing')==False].reset_index(drop=True)
questions

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
206362,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,2000,Turandot,Double Jeopardy!,4999
206363,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,2000,a titmouse,Double Jeopardy!,4999
206364,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",2000,Clive Barker,Double Jeopardy!,4999
206365,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",2000,Geronimo,Double Jeopardy!,4999


In [17]:
nltk.download('punkt')
nltk.download('stopwords')
# skipgram and cbow

# preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [re.sub(r'[^a-zA-Z0-9]', '', token) for token in tokens if token not in stop_words]
    return tokens

# Preprocess and tokenize Jeopardy questions
preprocessed_questions = [(preprocess_text(question), category) for question, category in zip(questions['question'], questions['category'])][:100000]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amary\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amary\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Train Skip-gram model
sentences = [tokens for tokens, _ in preprocessed_questions]
skipgram_model = Word2Vec(sentences, sg=1, vector_size=100, window=5, min_count=1, workers=4)

# Generate embeddings for each question using Skip-gram
def generate_skipgram_embedding(tokens, model):
    embeddings = []
    for token in tokens:
        if token in model.wv.key_to_index:
            embeddings.append(model.wv[token])
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)  # If no embeddings found, return zeros
    return np.mean(embeddings, axis=0)  # Average embeddings for all tokens

In [19]:
# Train CBOW model
cbow_model = Word2Vec(sentences, sg=0, vector_size=100, window=5, min_count=1, workers=4)

# Generate embeddings for each question using CBOW
def generate_cbow_embedding(tokens, model):
    embeddings = []
    for token in tokens:
        if token in model.wv.key_to_index:
            embeddings.append(model.wv[token])
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)  # If no embeddings found, return zeros
    return np.mean(embeddings, axis=0)  # Average embeddings for all tokens


In [20]:
X_skipgram = np.array([generate_skipgram_embedding(tokens, skipgram_model) for tokens, _ in preprocessed_questions])
X_cbow = np.array([generate_cbow_embedding(tokens, cbow_model) for tokens, _ in preprocessed_questions])
y = np.array([category for _, category in preprocessed_questions])

# Split data into train and test sets for Skip-gram
X_train_skipgram, X_test_skipgram, y_train, y_test = train_test_split(X_skipgram, y, test_size=0.2, random_state=42)

# Split data into train and test sets for CBOW
X_train_cbow, X_test_cbow, _, _ = train_test_split(X_cbow, y, test_size=0.2, random_state=42)


In [21]:
# Train a logistic regression classifier for Skip-gram
classifier_skipgram = LogisticRegression(max_iter=50)
classifier_skipgram.fit(X_train_skipgram, y_train)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=50)

In [23]:
classifier_cbow = LogisticRegression(max_iter=50)
classifier_cbow.fit(X_train_cbow, y_train)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=50)

In [24]:
# Predict on the test set for Skip-gram
y_pred_skipgram = classifier_skipgram.predict(X_test_skipgram)


In [25]:
# Predict on the test set for CBOW
y_pred_cbow = classifier_cbow.predict(X_test_cbow)

In [26]:
# Evaluate accuracy for Skip-gram
accuracy_skipgram = accuracy_score(y_test, y_pred_skipgram)
print("Accuracy for Skip-gram:", accuracy_skipgram)

# Evaluate accuracy for CBOW
accuracy_cbow = accuracy_score(y_test, y_pred_cbow)
print("Accuracy for CBOW:", accuracy_cbow)

Accuracy for Skip-gram: 0.03105
Accuracy for CBOW: 0.0189
